In [ ]:
import sys
import json
import os
from typing import Dict
from pydantic import BaseModel, ValidationError
from itertools import islice

In [ ]:
# Модель данных для покупок
class UserData(BaseModel):
    user_id: str
    category: str

In [ ]:
def read_file_to_dict(file_path: str) -> Dict[str, str]:
    """
    Чтение файла и преобразование его содержимого в словарь.
    Args:
        file_path: путь к файлу
    Returns:
        dict with keys and values from the file.

    """

    # Словарь для хранения результатов
    output_dict = {}

    # Чтение файла с utf-8
    with open(file_path, "r", encoding="utf-8") as file:
        # Пропуск заголовка
        next(file)

        for line_num, line in enumerate(file, 2):
            try:
                # Преобразование строки в объект модели данных
                data = UserData(**json.loads(line.strip()))
                output_dict[data.user_id] = data.category
            except ValidationError as e:
                print(f"Error on line {line_num}: {e}")
            except json.JSONDecodeError:
                print(f"Error decoding JSON on line {line_num}.")

    return output_dict

In [ ]:
def first_two_items(d: Dict) -> Dict:
    """2 строки, как в задании."""
    return dict(islice(d.items(), 2))

In [ ]:
def run_homework():
    """Запуск, в т.ч. в тетрадке."""

    # Проверка на выполнение в Jupyter
    is_jupyter = 'ipykernel' in sys.modules
    if is_jupyter:
        DEFAULT_FILE_PATH = os.path.join(os.getcwd(), 'purchase_log.txt')
    else:
        DEFAULT_FILE_PATH = os.path.join(os.path.dirname(os.path.abspath(__file__)), 'purchase_log.txt')
    try:
        purchases = read_file_to_dict(DEFAULT_FILE_PATH)
        if not purchases:
            print("The file does not contain valid data entries after the header.")
        else:
            output_purchases = first_two_items(purchases)
            for key, value in output_purchases.items():
                print(f"{key} '{value}'")  # Кавычки, как в задании
    except FileNotFoundError:
        print(f"File '{DEFAULT_FILE_PATH}' not found.")

In [ ]:
if __name__ == "__main__":
    run_homework()